In [1]:
import pandas as pd

In [19]:


# Example DataFrame
data = {'REGION_NAME': ['CVIS', 'CVIS'],
        'RUN_TIME': ['1/17/2022 17:15', '1/17/2022 17:20'],
        'RGWAP_RU': [10846.35, 10990.00]}

df = pd.DataFrame(data)

# Convert RUN_TIME to datetime
df['RUN_TIME'] = pd.to_datetime(df['RUN_TIME'])

# Set the starting point for backfilling (January 1, 2022 00:00)
start_time = pd.Timestamp('2022-01-01 00:00:00')

# Initialize an empty list to store rows with inserted previous rows
new_rows = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Backfill missing times until we reach the current row's RUN_TIME
    current_time = row['RUN_TIME']
    region = row['REGION_NAME']
    
    while current_time > start_time:
        previous_time = current_time - pd.Timedelta(minutes=5)
        
        # Check if the previous time already exists in the DataFrame
        if not ((df['RUN_TIME'] == previous_time) & (df['REGION_NAME'] == region)).any():
            previous_row = {
                'REGION_NAME': region,
                'RUN_TIME': previous_time,
                'RGWAP_RU': 0
            }
            new_rows.append(previous_row)
        
        # Update current_time to previous_time for the next iteration
        current_time = previous_time
    
    # Add the current row
    new_rows.append(row)

# Convert the list of rows back to a DataFrame
new_df = pd.DataFrame(new_rows)

# Sort by REGION_NAME and RUN_TIME to maintain order
new_df = new_df.sort_values(by=['REGION_NAME', 'RUN_TIME']).reset_index(drop=True)
new_df.drop_duplicates(subset=['RUN_TIME'],inplace=True)
new_df = new_df.reset_index(drop=True)

print(new_df)


     REGION_NAME            RUN_TIME  Weighted_Avg_Price
0           CVIS 2022-01-01 00:00:00                0.00
1           CVIS 2022-01-01 00:05:00                0.00
2           CVIS 2022-01-01 00:10:00                0.00
3           CVIS 2022-01-01 00:15:00                0.00
4           CVIS 2022-01-01 00:20:00                0.00
...          ...                 ...                 ...
4812        CVIS 2022-01-17 17:00:00                0.00
4813        CVIS 2022-01-17 17:05:00                0.00
4814        CVIS 2022-01-17 17:10:00                0.00
4815        CVIS 2022-01-17 17:15:00            10846.35
4816        CVIS 2022-01-17 17:20:00            10990.00

[4817 rows x 3 columns]


In [22]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\ReserveGWAP_RD.csv'
data = pd.read_csv(input_file)
data=data[data['REGION_NAME']=='CMIN']
print(len(data))
duplicated_data = data[data.duplicated(subset=['RUN_TIME'], keep=False)]
print(f"Total duplicated rows: {len(duplicated_data)}")
data = data.drop_duplicates('RUN_TIME', keep='last')
data.dropna(inplace=True, how='all')
print(len(data))
print(duplicated_data)

210240
Total duplicated rows: 0
210240
Empty DataFrame
Columns: [REGION_NAME, RUN_TIME, GWAP_RD]
Index: []


In [11]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveGWAP_RD.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE', 'PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'NL' and COMMODITY_TYPE is 'Dr'
    chunk = chunk[(chunk['RESOURCE_TYPE'] == 'G') & (chunk['COMMODITY_TYPE'] == 'Rd')]
    
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Append processed chunk to the list
    chunks.append(chunk)

# Concatenate all processed chunks into one DataFrame
data = pd.concat(chunks, ignore_index=True)

# Group by REGION_NAME and RUN_TIME with 5-minute frequency
grouped = data.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

# Calculate the weighted average (LGWAP_DR) for each group
def calculate_weighted_avg(x):
    total_sched_mw = x['SCHED_MW'].sum()
    if total_sched_mw == 0:
        return pd.Series({'GWAP_RD': 0})  # Handle division by zero
    return pd.Series({
        'GWAP_RD': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
    })

# Apply the function to the group and exclude the grouping columns from the operation
result = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()


# Create a complete time index for each region
min_time = data['RUN_TIME'].min()
max_time = data['RUN_TIME'].max()
complete_time_index = pd.date_range(start=min_time, end=max_time, freq='5min')

# Reindex the DataFrame to include all time periods and fill missing values with 0
reindexed_chunks = []
for region in result['REGION_NAME'].unique():
    region_data = result[result['REGION_NAME'] == region]
    region_data = region_data.set_index('RUN_TIME').reindex(complete_time_index, fill_value=0).reset_index()
    region_data['REGION_NAME'] = region
    region_data.columns = ['REGION_NAME','RUN_TIME' , 'GWAP_RD']
    reindexed_chunks.append(region_data)

# Concatenate all reindexed chunks into one DataFrame
final_result = pd.concat(reindexed_chunks, ignore_index=True)

# Reorder columns to REGION, RUNTIME, Price
final_result = final_result[['REGION_NAME', 'RUN_TIME', 'GWAP_RD']]
final_result.columns = ['REGION_NAME', 'RUN_TIME', 'GWAP_RD']

# Write the result to the output file in chunks
final_result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [12]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveGWAP_RU.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE', 'PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'NL' and COMMODITY_TYPE is 'Dr'
    chunk = chunk[(chunk['RESOURCE_TYPE'] == 'G') & (chunk['COMMODITY_TYPE'] == 'Ru')]
    
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Append processed chunk to the list
    chunks.append(chunk)

# Concatenate all processed chunks into one DataFrame
data = pd.concat(chunks, ignore_index=True)

# Group by REGION_NAME and RUN_TIME with 5-minute frequency
grouped = data.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

# Calculate the weighted average (LGWAP_DR) for each group
def calculate_weighted_avg(x):
    total_sched_mw = x['SCHED_MW'].sum()
    if total_sched_mw == 0:
        return pd.Series({'GWAP_RU': 0})  # Handle division by zero
    return pd.Series({
        'GWAP_RU': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
    })

# Apply the function to the group and exclude the grouping columns from the operation
result = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()


# Create a complete time index for each region
min_time = data['RUN_TIME'].min()
max_time = data['RUN_TIME'].max()
complete_time_index = pd.date_range(start=min_time, end=max_time, freq='5min')

# Reindex the DataFrame to include all time periods and fill missing values with 0
reindexed_chunks = []
for region in result['REGION_NAME'].unique():
    region_data = result[result['REGION_NAME'] == region]
    region_data = region_data.set_index('RUN_TIME').reindex(complete_time_index, fill_value=0).reset_index()
    region_data['REGION_NAME'] = region
    region_data.columns = ['REGION_NAME','RUN_TIME' , 'GWAP_RU']
    reindexed_chunks.append(region_data)

# Concatenate all reindexed chunks into one DataFrame
final_result = pd.concat(reindexed_chunks, ignore_index=True)

# Reorder columns to REGION, RUNTIME, Price
final_result = final_result[['REGION_NAME', 'RUN_TIME', 'GWAP_RU']]
final_result.columns = ['REGION_NAME', 'RUN_TIME', 'GWAP_RU']

# Write the result to the output file in chunks
final_result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [27]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveLWAP_DR.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'NL' and COMMODITY_TYPE is 'Dr'
    chunk = chunk[(chunk['RESOURCE_TYPE'] == 'G') & (chunk['COMMODITY_TYPE'] == 'Dr')]
    
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'RGWAP_DR': 0})  # Handle division by zero
        return pd.Series({
            'RGWAP_DR': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })
    
    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Create a complete time index for each region
    min_time = chunk['RUN_TIME'].min()
    max_time = chunk['RUN_TIME'].max()
    complete_time_index = pd.date_range(start=min_time, end=max_time, freq='5min')

    # Reindex the DataFrame to include all time periods and fill missing values with 0
    reindexed_chunks = []
    for region in weighted_avg['REGION_NAME'].unique():
        region_data = weighted_avg[weighted_avg['REGION_NAME'] == region]
        region_data = region_data.set_index('RUN_TIME').reindex(complete_time_index, fill_value=0).reset_index()
        region_data['REGION_NAME'] = region
        region_data.columns = ['RUN_TIME', 'RGWAP_DR', 'REGION_NAME']
        reindexed_chunks.append(region_data)

    # Append processed chunk to the list
    chunks.extend(reindexed_chunks)
result = pd.concat(chunks, ignore_index=True)

result = result[['REGION_NAME', 'RUN_TIME', 'RGWAP_DR']]
result.columns = ['REGION', 'RUNTIME', 'RGWAP_DR']
# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [26]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveLWAP_RU.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'NL' and COMMODITY_TYPE is 'Dr'
    chunk = chunk[(chunk['RESOURCE_TYPE'] == 'G') & (chunk['COMMODITY_TYPE'] == 'Ru')]
    
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'GWAP_RU': 0})  # Handle division by zero
        return pd.Series({
            'GWAP_RU': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })
    
    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Create a complete time index for each region
    min_time = chunk['RUN_TIME'].min()
    max_time = chunk['RUN_TIME'].max()
    complete_time_index = pd.date_range(start=min_time, end=max_time, freq='5min')

    # Reindex the DataFrame to include all time periods and fill missing values with 0
    reindexed_chunks = []
    for region in weighted_avg['REGION_NAME'].unique():
        region_data = weighted_avg[weighted_avg['REGION_NAME'] == region]
        region_data = region_data.set_index('RUN_TIME').reindex(complete_time_index, fill_value=0).reset_index()
        region_data['REGION_NAME'] = region
        region_data.columns = ['RUN_TIME', 'GWAP_RU', 'REGION_NAME']
        reindexed_chunks.append(region_data)

    # Append processed chunk to the list
    chunks.extend(reindexed_chunks)
result = pd.concat(chunks, ignore_index=True)

result = result[['REGION_NAME', 'RUN_TIME', 'GWAP_RU']]
result.columns = ['REGION', 'RUNTIME', 'GWAP_RU']
# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [3]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveLWAP_DR.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'G'
    chunk = chunk[chunk['RESOURCE_TYPE'] == 'NL']
    chunk = chunk[chunk['COMMODITY_TYPE'] == 'Dr']
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'LGWAP_DR': 0})  # Handle division by zero
        return pd.Series({
            'LGWAP_DR': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })

    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Append processed chunk to the list
    chunks.append(weighted_avg)

# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [23]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'Reserve_FR.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'G'
    chunk = chunk[chunk['RESOURCE_TYPE'] == 'G']
    chunk = chunk[chunk['COMMODITY_TYPE'] == 'Fr']
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'RLWAP_FR': 0})  # Handle division by zero
        return pd.Series({
            'RLWAP_FR': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })

    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Append processed chunk to the list
    chunks.append(weighted_avg)

# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [13]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'Reserve_RU.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'G'
    chunk = chunk[chunk['RESOURCE_TYPE'] == 'G']
    chunk = chunk[chunk['COMMODITY_TYPE'] == 'Ru']
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'RLWAP_RU': 0})  # Handle division by zero
        return pd.Series({
            'RLWAP_RU': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })

    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Append processed chunk to the list
    chunks.append(weighted_avg)

# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.


In [16]:
input_file = r'D:\School\ADMU\4Y\SEM 1\MATH 199.11\Final\Reserve_Complete.csv'
output_file = 'ReserveGWAP_RD.csv'

# Define chunk size (adjust based on your system's capacity)
chunk_size = 500000

# Columns to retain (drop unnecessary columns during reading)
usecols = ['RUN_TIME', 'RESOURCE_TYPE', 'REGION_NAME', 'COMMODITY_TYPE','PRICE', 'SCHED_MW']

# Initialize an empty list to hold processed chunks
chunks = []

# Read and process file in chunks
for chunk in pd.read_csv(input_file, usecols=usecols, chunksize=chunk_size, parse_dates=['RUN_TIME']):
    # Filter rows where RESOURCE_TYPE is 'G'
    chunk = chunk[chunk['RESOURCE_TYPE'] == 'G']
    chunk = chunk[chunk['COMMODITY_TYPE'] == 'Rd']
    # Set negative LMP prices to 0
    chunk.loc[chunk['PRICE'] < 0, 'PRICE'] = 0
    
    # Group by REGION_NAME and RUN_TIME with 5-minute frequency
    grouped = chunk.groupby(['REGION_NAME', pd.Grouper(key='RUN_TIME', freq='5min')])

    # Calculate the weighted average (GWAP) for each group
    def calculate_weighted_avg(x):
        total_sched_mw = x['SCHED_MW'].sum()
        if total_sched_mw == 0:
            return pd.Series({'RGWAP_RD': 0})  # Handle division by zero
        return pd.Series({
            'RGWAP_RD': (x['PRICE'] * x['SCHED_MW']).sum() / total_sched_mw
        })

    # Apply the function to the group and exclude the grouping columns from the operation
    weighted_avg = grouped.apply(calculate_weighted_avg, include_groups=False).reset_index()

    # Append processed chunk to the list
    chunks.append(weighted_avg)

# Concatenate all processed chunks into one DataFrame
result = pd.concat(chunks, ignore_index=True)

# Write the result to the output file in chunks
result.to_csv(output_file, index=False, chunksize=50000)

print("Processing complete.")

Processing complete.
